In [ ]:
using PyPlot, Jets, JetPack, JetPackDSP

## Read in model from binary

In [ ]:
nz,nx = 176,851
v = read!("marmousi_vp_20m_176x851.bin", Array{Float32}(undef, nz,nx));

## Smooth model using JetPackDSP convolutional smoother

In [ ]:
?JetPackDSP.JopConvolve

In [ ]:
a = 10
b = a+1
P = JopPad(JetSpace(Float64,nz,nx), -b:nz+b, -b:nx+b, extend=true)
S = JopConvolve(range(P), smoother=:rect, n=(a,a))
R = JopPad(JetSpace(Float64,nz,nx), -b:nz+b, -b:nx+b, extend=false)
slowness_sm = R'∘S∘P*(1 ./v)
vsm = 1 ./(slowness_sm);

In [ ]:
figure(figsize=(20,5))
subplot(121);imshow(v,cmap="jet",aspect="auto");clim(1500,4500);title("True model");
subplot(122);imshow(vsm,cmap="jet",aspect="auto");clim(1500,4500);title("Smoothed model");

## Next we find the water layer in the original model and paste it on the smoothed model

In [ ]:
vsm[v.==1500.0].=1500.0;

In [ ]:
figure(figsize=(20,5))
subplot(121);imshow(v,cmap="jet",aspect="auto");clim(1500,4500);title("True model");
subplot(122);imshow(vsm,cmap="jet",aspect="auto");clim(1500,4500);title("Smoothed model");

## Write out the smoothed model for later

In [ ]:
write("marmousi_vp_smooth_20m_176x851.bin",Float32.(vsm))